In [187]:
#Import necessary libraries
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter


In [188]:

# Set your OpenAI API key
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY'

In [189]:
data = [
    "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
    "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
    "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
    "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
    "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
    "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
    "7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
    "8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
    "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
    "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
    "11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
    "12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
    "13: User: Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
    "14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
    "15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
    "16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
    "17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
    "18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
    "19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.",
    "20: User: Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences."
]

In [190]:
#Create a function to generate embeddings and store in FAISS
def create_vectorstore(data):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(data[-20:], embedding=embeddings)
    return vectorstore

In [191]:
#Define the prompt template
template = """
You are an AI assistant. Use the following context (delimited by <ctx></ctx>) to answer the question:
You should only answer based on the stored context. If a question is outside the stored context, reply with 'I don't have information on that topic.'
<ctx>
{context}
</ctx>
------
<hs>
{chat_history}
</hs>
------
{question}
Answer:
"""

prompt = PromptTemplate(input_variables=["chat_history", "context", "question"], template=template)


In [192]:
#Create the conversation chain
def get_conversation_chain(vectorstore):
    llm = ChatOpenAI(model_name="gpt-4")  # You can choose other models too
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer')
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        combine_docs_chain_kwargs={'prompt': prompt},
        memory=memory,
        return_source_documents=True,
        verbose=True
    )
    return conversation_chain

In [193]:
#Create the vector store and conversation chain
vectorstore = create_vectorstore(data)
conversation_chain = get_conversation_chain(vectorstore)

In [194]:
def handle_user_input(user_question):
    response = conversation_chain({"question": user_question})
    answer = response['answer']
    chat_history = response['chat_history']

    source_documents = response.get('source_documents', [])
    print("Source Documents:")
    for doc in source_documents:
        print(doc.page_content)

    # total tokens (prompt + context + answer) do not exceed 225
    total_tokens = len(prompt.format(chat_history=chat_history, context='', question=user_question).split()) + len(answer.split())
    if total_tokens > 225:
        return "The response is too long. Please ask a more specific question.", chat_history

    return answer, chat_history

In [196]:
# Test the chat application
user_question = "any speacific?"
answer, chat_history = handle_user_input(user_question)
print("Chat History:")
for message in chat_history:
    print(message.content)
print("Answer:", answer)

Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: what are some good exercises?
Assistant: You can try a mix of cardio and strength training exercises.
Follow Up Input: any speacific?
Standalone question:



> Finished chain.
Prompt after formatting:

You are an AI assistant. Use the following context (delimited by <ctx></ctx>) to answer the question:
You should only answer based on the stored context. If a question is outside the stored context, reply with 'I don't have information on that topic.'
<ctx>
8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.

7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.

9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.

10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.
</ctx>
------
<hs>

Human: what are some good exercises?
Assistant: You can try a mix of cardio and strength training exercises.
</hs>
------
Can you recommend any specific exercises?
Answ